In [79]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import cross_val_score
from catboost import CatBoostClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
pd.options.display.max_columns = 50
plt.rcParams["figure.figsize"] = (20,12)

In [80]:
train = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\train.csv")
test = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\test.csv")
alldata = pd.concat([train,test])
alldata

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
0,0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0.0
1,1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1.0
2,2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0.0
3,3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0.0
4,4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,399995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.185843,1.186322,1.185493,1.185652,1.186067,NaN
79996,399996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,1.284956,1.290772,1.279329,1.282120,1.287838,NaN
79997,399997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.188475,1.188563,1.188518,1.188480,1.188503,NaN
79998,399998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.180429,1.181329,1.179654,1.180025,1.180862,NaN


In [81]:
# 이상치 제거 1 - 분포

# train = train.drop(train[train["u"] < -10].index) # 애매한데
# train = train.drop(train[(train["g"] < -15000)].index) 
# train = train.drop(train[train["r"] > 60].index)
# train = train.drop(train[train["i"] < -20000].index)
# train = train.drop(train[train["z"] < -6000].index)

# plt.figure(figsize=(20,12))
# sns.boxplot(train["class"], train["u"])
# sns.countplot(alldata["nObserve"], alldata["class"])


# # 이상치 제거 2 - 상관도
# train = train.drop(train[(train["g"] > 80) | (train["g"] < -40)].index)
# train = train.drop(train[train["redshift"] < -100].index)
# train = train.drop(train[train["z"] > 6000].index)
# train = train.drop(train[(train["redshift"] < -40) & (train["nObserve"] > 5)].index)
# train
# plt.figure(figsize=(20,12))
# sns.scatterplot(alldata["i"], alldata["z"])
# train

In [82]:
# plt.figure(figsize = (20,12))
# sns.heatmap(train.corr(), annot = True, annot_kws = {"size" : 11}, linewidths = 1 )

망원경에서 망원경으로 또는 밤에서 밤까지 절대교정과 관련된 불확실성을 줄이기 위해 

천문학자들은 일반적으로 두 개의 서로 다른 필터 밴드 사이의 크기 차이로 정의되는 색상으로 작업한다. 

두 개의 크기를 빼면 이러한 불확실성이 감소한다. 

예를 들어, 위의 별 베가에 대한 관측은 벡터 4개의 숫자로 구성된다. [u-g, g-r, r-i, i-z]

In [83]:
alldata["u-g"] = alldata["u"]-alldata["g"]
alldata["g-r"] = alldata["g"]-alldata["r"]
alldata["r-i"] = alldata["r"]-alldata["i"]
alldata["i-z"] = alldata["i"]-alldata["z"]

alldata["d1"] = alldata["dered_u"]-alldata["dered_g"]
alldata["d2"] = alldata["dered_g"]-alldata["dered_r"]
alldata["d3"] = alldata["dered_r"]-alldata["dered_i"]
alldata["d4"] = alldata["dered_i"]-alldata["dered_z"]

alldata["minusOD"] = alldata["nObserve"] - alldata["nDetect"]
alldata["plusOD"] = alldata["nObserve"] + alldata["nDetect"]

# alldata["nObserve"] = np.log1p(alldata["nObserve"])
# alldata["nDetect"] = np.log1p(alldata["nDetect"])


alldata

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u-g,g-r,r-i,i-z,d1,d2,d3,d4,minusOD,plusOD
0,0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0.0,2.927183,1.327273,1.337061,0.732833,2.86647,1.30267,1.32301,0.72317,0,36
1,1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1.0,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157,0,2
2,2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0.0,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565,0,4
3,3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0.0,4.471910,0.967569,0.326209,0.260300,4.35900,0.90738,0.29262,0.23947,1,7
4,4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0.0,3.754259,1.656803,0.718216,0.341503,3.33756,1.57085,0.67271,0.31436,1,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,399995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.185843,1.186322,1.185493,1.185652,1.186067,NaN,1.011727,0.390873,0.148287,0.059829,0.94283,0.30731,0.09990,0.01911,0,34
79996,399996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,1.284956,1.290772,1.279329,1.282120,1.287838,NaN,0.794588,0.119586,-0.396179,0.159076,0.75943,0.09486,-0.40563,0.16801,4,12
79997,399997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.188475,1.188563,1.188518,1.188480,1.188503,NaN,1.157989,0.080584,-0.037262,-0.004078,1.03826,-0.02936,-0.09957,-0.03574,0,2
79998,399998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.180429,1.181329,1.179654,1.180025,1.180862,NaN,1.298879,0.467991,0.310531,0.227785,1.28355,0.43231,0.28972,0.20745,0,34


In [84]:
alldata["binning_observe"] = pd.cut(alldata['nObserve'], 10, labels=False)

alldata

,id,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u-g,g-r,r-i,i-z,d1,d2,d3,d4,minusOD,plusOD,binning_observe
0,0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,1.189764,1.190681,1.188979,1.189355,1.190206,0.0,2.927183,1.327273,1.337061,0.732833,2.86647,1.30267,1.32301,0.72317,0,36,3
1,1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,1.253254,1.257836,1.248762,1.250997,1.255533,1.0,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157,0,2,0
2,2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,1.022499,1.024105,1.020983,1.021730,1.023291,0.0,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565,0,4,0
3,3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,1.205399,1.206058,1.204874,1.205120,1.205712,0.0,4.471910,0.967569,0.326209,0.260300,4.35900,0.90738,0.29262,0.23947,1,7,0
4,4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,1.193946,1.194285,1.193738,1.193826,1.194099,0.0,3.754259,1.656803,0.718216,0.341503,3.33756,1.57085,0.67271,0.31436,1,25,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,399995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.185843,1.186322,1.185493,1.185652,1.186067,NaN,1.011727,0.390873,0.148287,0.059829,0.94283,0.30731,0.09990,0.01911,0,34,3
79996,399996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,1.284956,1.290772,1.279329,1.282120,1.287838,NaN,0.794588,0.119586,-0.396179,0.159076,0.75943,0.09486,-0.40563,0.16801,4,12,1
79997,399997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.188475,1.188563,1.188518,1.188480,1.188503,NaN,1.157989,0.080584,-0.037262,-0.004078,1.03826,-0.02936,-0.09957,-0.03574,0,2,0
79998,399998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.180429,1.181329,1.179654,1.180025,1.180862,NaN,1.298879,0.467991,0.310531,0.227785,1.28355,0.43231,0.28972,0.20745,0,34,3


In [85]:
alldata2 = alldata.drop(["id","class"], axis=1)
alldata2 = alldata2.drop(["airmass_u","airmass_g","airmass_r","airmass_i","airmass_z"], axis=1 )

alldata2

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,u-g,g-r,r-i,i-z,d1,d2,d3,d4,minusOD,plusOD,binning_observe
0,23.263956,20.336773,19.009500,17.672439,16.939607,-0.000081,23.12426,20.25779,18.95512,17.63211,16.90894,18,18,2.927183,1.327273,1.337061,0.732833,2.86647,1.30267,1.32301,0.72317,0,36,3
1,15.052147,14.061969,13.452418,13.268450,13.168941,0.004506,14.96642,14.00452,13.41139,13.23625,13.13468,1,1,0.990179,0.609551,0.183968,0.099509,0.96190,0.59313,0.17514,0.10157,0,2,0
2,16.786385,15.825426,15.536318,15.393535,15.350032,0.000472,16.60765,15.68659,15.44004,15.32173,15.29608,2,2,0.960959,0.289108,0.142783,0.043503,0.92106,0.24655,0.11831,0.02565,0,4,0
3,25.660638,21.188727,20.221158,19.894949,19.634649,0.000006,25.35365,20.99465,20.08727,19.79465,19.55518,4,3,4.471910,0.967569,0.326209,0.260300,4.35900,0.90738,0.29262,0.23947,1,7,0
4,24.453429,20.699170,19.042368,18.324152,17.982649,-0.000033,23.77140,20.43384,18.86299,18.19028,17.87592,13,12,3.754259,1.656803,0.718216,0.341503,3.33756,1.57085,0.67271,0.31436,1,25,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,19.673388,18.661662,18.270788,18.122501,18.062672,-0.000317,19.33614,18.39331,18.08600,17.98610,17.96699,17,17,1.011727,0.390873,0.148287,0.059829,0.94283,0.30731,0.09990,0.01911,0,34,3
79996,21.522869,20.728282,20.608695,21.004875,20.845799,0.065455,21.39935,20.63992,20.54506,20.95069,20.78268,8,4,0.794588,0.119586,-0.396179,0.159076,0.75943,0.09486,-0.40563,0.16801,4,12,1
79997,19.399065,18.241076,18.160492,18.197753,18.201831,-0.000842,18.91359,17.87533,17.90469,18.00426,18.04000,1,1,1.157989,0.080584,-0.037262,-0.004078,1.03826,-0.02936,-0.09957,-0.03574,0,2,0
79998,18.899747,17.600868,17.132878,16.822347,16.594561,0.042817,18.77153,17.48798,17.05567,16.76595,16.55850,17,17,1.298879,0.467991,0.310531,0.227785,1.28355,0.43231,0.28972,0.20745,0,34,3


In [86]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [87]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import classification_report, accuracy_score

# x_train,x_valid, y_train, y_valid = train_test_split(train2, train["class"], test_size = 0.2, random_state = 42, shuffle = True)

# hr2_accuracy_list = []

# max_depth_list = [180,190,200,210,220]
# # subsample_list = [0.6,0.7,0.8,0.9,1.0]

# def hr2_grid_search(max_depth): 
#     xgb_model = LGBMClassifier(n_estimators = 200, num_leaves=max_depth, n_jobs = -1)
#     xgb_pred = xgb_model.fit(x_train, y_train).predict(x_valid)
#     return([max_depth, accuracy_score(y_valid, xgb_pred)])
      

# for max_depth in max_depth_list:
#     hr2_accuracy_list.append(hr2_grid_search(max_depth))
    

# hr2_xgb_df = pd.DataFrame(hr2_accuracy_list, columns=['tree depth', 'accuracy'])
# print("accuracy max : "+str(hr2_xgb_df["accuracy"].argmax())+"행") 
# print(hr2_xgb_df)

In [ ]:
## randomforest모델
# rf = RandomForestClassifier(n_estimators = 400)
# rf.fit(train2,train["class"])
# rf_result = rf.predict_proba(test2)

# rf_df = pd.DataFrame(rf_result)
# rf_df.columns = ["rf1","rf2","rf3"]
# rf_df.to_csv("rf_result.csv", index = False)


## catboost모델
# ca = CatBoostClassifier(n_estimators = 2000)
# ca.fit(x_train,y_train, eval_set = (x_valid, y_valid), early_stopping_rounds = 30)


x_train, x_valid, y_train, y_valid = train_test_split(train2,train["class"], 
                                                      test_size = 0.2, 
                                                      random_state = 42, 
                                                      shuffle = True, 
                                                      stratify = train["class"])

# lgb모델
# 1554 = n_estimators = 1500, num_leaves = 200, n_jobs = -1, learning_rate = 0.01

lgb = LGBMClassifier(n_estimators = 300, num_leaves = 200, n_jobs = -1)
lgb.fit(x_train,y_train, eval_set = (x_valid, y_valid), early_stopping_rounds = 100)
# result = lgb.predict_proba(x_valid)

# 200 0.1557 -> 0.1554


[1]	valid_0's multi_logloss: 0.874324
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.784338
[3]	valid_0's multi_logloss: 0.709291
[4]	valid_0's multi_logloss: 0.645802
[5]	valid_0's multi_logloss: 0.591262
[6]	valid_0's multi_logloss: 0.54422
[7]	valid_0's multi_logloss: 0.503247
[8]	valid_0's multi_logloss: 0.467315
[9]	valid_0's multi_logloss: 0.435682
[10]	valid_0's multi_logloss: 0.407831
[11]	valid_0's multi_logloss: 0.383158
[12]	valid_0's multi_logloss: 0.361098
[13]	valid_0's multi_logloss: 0.341515
[14]	valid_0's multi_logloss: 0.324005
[15]	valid_0's multi_logloss: 0.308306
[16]	valid_0's multi_logloss: 0.294264
[17]	valid_0's multi_logloss: 0.281649
[18]	valid_0's multi_logloss: 0.270372
[19]	valid_0's multi_logloss: 0.260188
[20]	valid_0's multi_logloss: 0.251022
[21]	valid_0's multi_logloss: 0.242752
[22]	valid_0's multi_logloss: 0.235276
[23]	valid_0's multi_logloss: 0.228483
[24]	valid_0's multi_logloss: 0.222399
[25]	valid_0

In [ ]:
# #####  Kfold 제출용 
# sfk = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
# result = 0

# for i,j in sfk.split(train2,train["class"]):
#     x_train = train2.iloc[i]
#     x_valid = train2.iloc[j]
#     y_train = train["class"].iloc[i]
#     y_valid = train["class"].iloc[j]

#     lgb = LGBMClassifier(n_estimators = 300, num_leaves = 200, n_jobs = -1)
#     lgb.fit(x_train,y_train, eval_set = (x_valid, y_valid), early_stopping_rounds = 150)
#     result += lgb.predict_proba(test2)/10
    
# lgb_result = pd.DataFrame(result)
# lgb_result.columns = ["lgb1","lgb2","lgb3"]
# lgb_result.to_csv("lgb_result.csv", index = False)



In [ ]:
# print(accuracy_score(np.array(y_valid).astype(int),result.argmax(1)))
# confusion_matrix(np.array(y_valid).astype(int),result.argmax(1)) 

In [ ]:
sub = pd.read_csv(r"C:\Users\scott\Desktop\dankook_dacon\dataset\sample_submission.csv")
sub

In [ ]:
sub["class"] = result.argmax(1)
sub

In [ ]:
sub.to_csv("dk_dacon.csv", index=False)

In [ ]:
pd.Series(lgb.feature_importances_, index = train2.columns).sort_values(ascending = False).reset_index()